In [1]:
#allows drawing digrams in jupyeter notebooks
%matplotlib inline
#loading tensorboard
%load_ext tensorboard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#specific tensorflow libraries
import tensorflow as tf
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
import datetime

In [4]:
from pandas.plotting import scatter_matrix

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [6]:
def Train_Test_XY_normalize_PCA(TrainX, TrainY, PCA_n = None):
    
    # normalize data
    scaler_new = StandardScaler()
    scaler_new.fit(TrainX)
    X_train_scale = scaler_new.transform(TrainX)
    print(f'X shape {X_train_scale.shape}')
    
    if PCA_n != None:
        # dimension reduction
        TrainX_PCA = Train_Test_XY_PCA(X_train_scale, PCA_n = PCA_n)
        print(f'X shape {TrainX_PCA.shape}')
    
        # normalize again
        scaler_new = StandardScaler()
        scaler_new.fit(TrainX_PCA)
        X_train_scale = scaler_new.transform(TrainX_PCA)
        
    
    X_train, X_test, y_train, y_test = train_test_split(X_train_scale,TrainY,
                                                       test_size=.2,
                                                       shuffle=True,
                                                       random_state=2020,
                                                       stratify=TrainY)
    
    
    return X_train, X_test, y_train, y_test

In [7]:
import tensorflow.keras.backend as Kb

# Write a custom loss function
def custom_loss(y_true, y_pred):
    binary_crossentropy = Kb.mean(Kb.binary_crossentropy(y_true, y_pred), axis = -1)
    prob_constraint = Kb.square(Kb.sum(y_pred, axis = -1)
                                - Kb.sum(y_true, axis = -1))

    return(binary_crossentropy+prob_constraint)

In [14]:
def NN_TF_model(structure, ipt_dim, learning_rate=0.01):
    model=Sequential()

    model.add(layers.Dense(structure[0], input_shape=(ipt_dim,), activation='relu'))
    
    for i in range(len(structure)-1):
        model.add(layers.Dense(structure[i+1], activation='relu'))
        
    model.add(layers.Dense(1, activation='sigmoid'))

    # opt = optimizers.SGD(learning_rate=0.01)
    # opt = optimizers.Adam(learning_rate=0.1)
    opt = optimizers.Nadam(learning_rate=0.01)

    # model.compile(loss='mean_squared_error', optimizer=opt)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    # model.compile(loss=custom_loss, optimizer=opt)
    # model.compile(loss='huber', optimizer=opt)
    # model.compile(loss='binary_focal_crossentropy', optimizer=opt)
    print(model.summary())
    
    return model

# simple NN model

In [15]:
# see_column = np.array(['Rk', 'AvAge', 'GP', 'W', 'L', 'OL', 'PTS', 'PTS%',
#        'GF', 'GA', 'SOW', 'SOL', 'SRS', 'SOS', 'GF/G', 'GA/G', 'PP', 'PPO',
#        'PP%', 'PPA', 'PPOA', 'PK%', 'SH', 'SHA', 'PIM/G', 'oPIM/G', 'S', 'S%',
#        'SA', 'SV%', 'SO', 'Hits', 'Hits/60', 'BkS', 'BkS/60', 'GvA', 'GvA/60',
#        'TkA', 'TkA/60', 'ENG', 'MsS', '5v5 TOI/GP', 'SAT%', 'Playoffs%',
#        'Playoffs', 'WonCup', 'TA/GA'])

see_column = np.array(['PTS', 'W','L', 'GF', 'GA', 'SRS', 
                                  'SOS', 'PK%', 'S%', 'SV%', 'SAT%', 
                                  'MsS', 'TA/GA'])
# see_column = np.array(['W', 'L', 'GA', 'SAT%', 'TA/GA'])

see_column2 = np.array(['Winner',
                       'F_home_assists', 'F_home_goals', 'F_home_pim', 'F_home_shots',
                       'F_home_blocked', 'F_home_pm', 'D_home_assists', 'D_home_goals',
                       'D_home_pim', 'D_home_shots', 'D_home_blocked', 'D_home_hits',
                       'D_home_pm', 'G_home_GAA', 'F_away_assists', 'F_away_goals',
                       'F_away_pim', 'F_away_shots', 'F_away_blocked', 'F_away_hits',
                       'F_away_pm', 'D_away_assists', 'D_away_goals', 'D_away_pim',
                       'D_away_shots', 'D_away_blocked', 'D_away_hits', 'D_away_pm',
                       'G_away_GAA', 'F_home_hits'])


column_home = np.array([i+'_H' for i in see_column])
column_away = np.array([i+'_A' for i in see_column])

column_name = np.concatenate((column_home,column_away))

column_name = np.concatenate((see_column2,column_name))
#  column_name = np.concatenate((column_name,['class'])) # Winner is the class

print(column_name)
print(len(column_name))

['Winner' 'F_home_assists' 'F_home_goals' 'F_home_pim' 'F_home_shots'
 'F_home_blocked' 'F_home_pm' 'D_home_assists' 'D_home_goals' 'D_home_pim'
 'D_home_shots' 'D_home_blocked' 'D_home_hits' 'D_home_pm' 'G_home_GAA'
 'F_away_assists' 'F_away_goals' 'F_away_pim' 'F_away_shots'
 'F_away_blocked' 'F_away_hits' 'F_away_pm' 'D_away_assists'
 'D_away_goals' 'D_away_pim' 'D_away_shots' 'D_away_blocked' 'D_away_hits'
 'D_away_pm' 'G_away_GAA' 'F_home_hits' 'PTS_H' 'W_H' 'L_H' 'GF_H' 'GA_H'
 'SRS_H' 'SOS_H' 'PK%_H' 'S%_H' 'SV%_H' 'SAT%_H' 'MsS_H' 'TA/GA_H' 'PTS_A'
 'W_A' 'L_A' 'GF_A' 'GA_A' 'SRS_A' 'SOS_A' 'PK%_A' 'S%_A' 'SV%_A' 'SAT%_A'
 'MsS_A' 'TA/GA_A']
57


#### Train NN model

In [16]:
tm_players = pd.read_csv('tm_player_stats_2005_2021.csv')

tm_players[column_name].sample(1)

,Winner,F_home_assists,F_home_goals,F_home_pim,F_home_shots,F_home_blocked,F_home_pm,D_home_assists,D_home_goals,D_home_pim,...,GF_A,GA_A,SRS_A,SOS_A,PK%_A,S%_A,SV%_A,SAT%_A,MsS_A,TA/GA_A
6915,0,0.969888,0.694463,2.075301,7.744782,2.213562,-0.252251,0.698632,0.282269,1.471373,...,2.426829,2.646341,-0.28,0.01,84.33,8.8,0.914,47.4,10.04878,0.795011


In [17]:
# Traindf_diff = pd.DataFrame(data = Traindf[column_home].to_numpy() 
#                                     - Traindf[column_away].to_numpy(), 
#                             columns = see_column)

Traindf = tm_players[column_name]
TrainX_array = Traindf[column_name[1:]].to_numpy()
TrainY_array = Traindf[column_name[0]].to_numpy()
print(sum(TrainY_array)/len(TrainY_array))

0.4239570498151734


In [18]:
X_train, X_test, y_train, y_test = Train_Test_XY_normalize_PCA(TrainX_array, 
                                                               TrainY_array)

X shape (11362, 56)


In [19]:
#creating unique name for tensorboard directory
log_dir = "logs/NN/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#Tensforboard callback function
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

struct = [20,10,30]
dim = X_train.shape[1]
model = NN_TF_model(structure = struct, ipt_dim = dim)
model.fit(X_train,
          y_train,
          epochs=1000,
          batch_size=100,
          shuffle=True,
          callbacks=[tensorboard_callback])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                1140      
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 30)                330       
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,711
Trainable params: 1,711
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
91/91 [==============================] - 1s 2ms/step - loss: 0.6049
Epoch 2/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.4786
Epoch 3/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0348
Epoch 90/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0407
Epoch 91/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.1097
Epoch 92/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 93/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0482
Epoch 94/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0563
Epoch 95/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0515
Epoch 96/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0406
Epoch 97/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0358
Epoch 98/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0355
Epoch 99/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0371
Epoch 100/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0354
Epoch 101/1000
91/91 [=======

91/91 [==============================] - 0s 1ms/step - loss: 0.0483
Epoch 189/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0525
Epoch 190/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0413
Epoch 191/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0329
Epoch 192/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0323
Epoch 193/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0316
Epoch 194/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0311
Epoch 195/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0348
Epoch 196/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0357
Epoch 197/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0570
Epoch 198/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0697
Epoch 199/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0445
Epoch 200/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0352
Epoch 287/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0345
Epoch 288/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0310
Epoch 289/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0368
Epoch 290/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0408
Epoch 291/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 292/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0700
Epoch 293/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0457
Epoch 294/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0327
Epoch 295/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0318
Epoch 296/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0319
Epoch 297/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0321
Epoch 298/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0567
Epoch 385/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0412
Epoch 386/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0408
Epoch 387/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0332
Epoch 388/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0322
Epoch 389/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0322
Epoch 390/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0314
Epoch 391/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0333
Epoch 392/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0317
Epoch 393/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0315
Epoch 394/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0322
Epoch 395/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0319
Epoch 396/1000
91/9

...

91/91 [==============================] - 0s 1ms/step - loss: 0.0328
Epoch 581/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0320
Epoch 582/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0319
Epoch 583/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0317
Epoch 584/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0378
Epoch 585/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0360
Epoch 586/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0358
Epoch 587/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0358
Epoch 588/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0407
Epoch 589/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0652
Epoch 590/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0626
Epoch 591/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0401
Epoch 592/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0320
Epoch 679/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0316
Epoch 680/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0317
Epoch 681/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0320
Epoch 682/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0317
Epoch 683/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0385
Epoch 684/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0474
Epoch 685/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0653
Epoch 686/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0452
Epoch 687/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0404
Epoch 688/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0352
Epoch 689/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0351
Epoch 690/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0308
Epoch 777/1000
91/91 [==============================] - 0s 2ms/step - loss: 0.0315
Epoch 778/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0315
Epoch 779/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0329
Epoch 780/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0310
Epoch 781/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0308
Epoch 782/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0400
Epoch 783/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0498
Epoch 784/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0582
Epoch 785/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0392
Epoch 786/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0327
Epoch 787/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0314
Epoch 788/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0322
Epoch 875/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0788
Epoch 876/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0418
Epoch 877/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0351
Epoch 878/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0324
Epoch 879/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0309
Epoch 880/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0308
Epoch 881/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0308
Epoch 882/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0379
Epoch 883/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0353
Epoch 884/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0321
Epoch 885/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0328
Epoch 886/1000
91/9

91/91 [==============================] - 0s 1ms/step - loss: 0.0570
Epoch 973/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0366
Epoch 974/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0387
Epoch 975/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0421
Epoch 976/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0313
Epoch 977/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0306
Epoch 978/1000
91/91 [==============================] - 0s 2ms/step - loss: 0.0311
Epoch 979/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0303
Epoch 980/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0302
Epoch 981/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0307
Epoch 982/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0303
Epoch 983/1000
91/91 [==============================] - 0s 1ms/step - loss: 0.0328
Epoch 984/1000
91/9

In [20]:
m = metrics.Accuracy()
mc = metrics.CategoricalAccuracy()

In [21]:
y_hat=model.predict(X_train)>0.5
y_hat=np.squeeze(y_hat)*1

285/285 [==============================] - 0s 598us/step


In [22]:
m.update_state(y_train, y_hat)
mc.update_state(y_train, y_hat)
print('away win rate = ', sum(y_hat)/len(y_hat))
print('Train data accuracy',m.result().numpy())
# print('Train data categorical accuracy',mc.result().numpy())
tf.math.confusion_matrix(y_train, y_hat)

away win rate =  0.4216085377929365
Train data accuracy 0.9827264


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[5168,   68],
       [  89, 3764]], dtype=int32)>

In [23]:
y_hat=model.predict(X_test)>0.5
y_hat=np.squeeze(y_hat)*1

72/72 [==============================] - 0s 646us/step


In [24]:
m.update_state(y_test, y_hat)
print('away win rate = ',sum(y_hat)/len(y_hat))
print('Test data accuracy',m.result().numpy())
tf.math.confusion_matrix(y_test, y_hat)

away win rate =  0.41970963484381874
Test data accuracy 0.98107725


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1285,   24],
       [  34,  930]], dtype=int32)>